# Parallel Computing with Ray: A Coffee Shop Story

Imagine you run a coffee shop. Orders come in throughout the day. You have multiple baristas behind the counter, each capable of making any drink. As drinks get made, your cash register keeps a running total of the day's revenue.

This is exactly how Ray thinks about distributed computing. You have work that can be done in parallel (making drinks), workers that can do it (baristas), and shared state that needs careful tracking (the cash register). Ray gives you a clean way to express this pattern in Python, whether you're running on a laptop or a thousand-machine cluster.

Let's build this system and see how Ray's pieces fit together.


In [ ]:
import ray
import ray.data as rd
import time
import logging

logging.getLogger("ray").setLevel(logging.WARNING)


/opt/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-09 19:45:19,146	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-12-09 19:45:19,518	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Opening the Shop: Ray clusters and workers

Before we can serve customers, we need to open the shop and staff it with baristas. In Ray terms, this means starting a cluster. When you call `ray.init()`, Ray spins up a head node (the manager coordinating everything) and a pool of worker processes (the baristas ready to work).

The `num_cpus` parameter tells Ray how many parallel tasks it can run at once. Think of it as hiring four baristas for the morning shift. Ray will schedule up to four pieces of work simultaneously across these workers.


In [2]:
ray.init(
    num_cpus=4,
    include_dashboard=False,
    logging_level=logging.WARNING,
    _system_config={"metrics_report_interval_ms": 0}
)


/opt/miniconda3/lib/python3.13/site-packages/ray/_private/worker.py:2062: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(


Python version:,3.13.5
Ray version:,2.52.1


In [3]:
print(f"Shop open with {ray.available_resources().get('CPU', 0)} baristas ready")
print(f"Running on {len(ray.nodes())} node(s)")


Shop open with 4.0 baristas ready
Running on 1 node(s)


## Making Drinks: Ray Tasks

Each drink order is independent work. A barista takes an order, makes the drink, and hands it back. No state to maintain, no coordination needed. This is what Ray calls a **task**: a stateless function that can run anywhere in the cluster.

You define a task by decorating a normal Python function with `@ray.remote`. When you call `make_drink.remote(order)`, Ray doesn't run it immediately. Instead, it schedules the work on an available worker and gives you back a reference to the future result. You only wait when you call `ray.get()`, which blocks until the result is ready.

This non-blocking design is key. You can fire off dozens of drink orders instantly, and Ray will parallelize them across your baristas without you managing threads or processes.


In [4]:
@ray.remote
def make_drink(order: dict) -> dict:
    """A barista makes a drink. Takes time, but no shared state."""
    drink_name = order["drink"]
    price = order["price"]
    
    # Simulate the time it takes to make a drink
    time.sleep(1)
    
    return {
        "drink": drink_name,
        "price": price,
        "status": "ready"
    }


## Tracking Revenue: Ray Actors

Unlike making drinks, tracking daily revenue requires state. You can't just have four separate cash registers. You need one centralized place that accumulates every sale. This is where Ray **actors** come in.

An actor is a Python class that Ray turns into a long-lived, stateful process living somewhere in the cluster. Every method call is routed to the same instance, executed serially, so there are no race conditions. You get stateful computation without worrying about locks or consistency.

Like tasks, actor method calls return immediately with a reference. The actual update happens asynchronously, but because all calls to the same actor are serialized, your state stays consistent.


In [5]:
@ray.remote
class CashRegister:
    """Tracks total revenue for the day. Stateful and centralized."""
    
    def __init__(self):
        self.total_revenue = 0.0
        self.drinks_sold = 0
    
    def record_sale(self, price: float) -> None:
        """Add a sale to today's total."""
        self.total_revenue += price
        self.drinks_sold += 1
    
    def get_summary(self) -> dict:
        """Return the current revenue summary."""
        return {
            "total_revenue": self.total_revenue,
            "drinks_sold": self.drinks_sold
        }


## Handling the Morning Rush: Ray Data

During the morning rush, orders flood in. You could manage them with Python lists, manually slicing and distributing work. But Ray Data gives you a cleaner abstraction: a distributed dataset that Ray can partition, stream, and iterate over efficiently.

You create a dataset from your orders, and Ray handles the sharding and distribution. When you iterate over it or map functions across it, Ray manages the parallel execution. It's the difference between manually coordinating work and letting the runtime do it for you.


In [6]:
# The morning rush: a batch of orders comes in
orders = [
    {"drink": "Cappuccino", "price": 4.50},
    {"drink": "Latte", "price": 5.00},
    {"drink": "Espresso", "price": 3.00},
    {"drink": "Americano", "price": 3.50},
    {"drink": "Mocha", "price": 5.50},
    {"drink": "Flat White", "price": 4.75},
]

# Create a Ray dataset from the orders
orders_dataset = rd.from_items(orders)


## Running the Shop

Now we put it all together. Orders come in through the dataset. We send each one to a barista (a Ray task). As drinks get made, we record each sale in the cash register (a Ray actor). Finally, we check the day's totals.

Notice the pattern: we submit work without waiting, collect references to the results, then block only when we actually need the values. This lets Ray parallelize aggressively while keeping our code simple and sequential-looking.


In [7]:
# Open the cash register for the day
register = CashRegister.remote()


In [8]:
# Process each order: send to baristas (tasks)
print("Processing orders...")
drink_refs = []

for order in orders_dataset.iter_rows():
    # Non-blocking: schedule the drink to be made
    drink_ref = make_drink.remote(order)
    drink_refs.append(drink_ref)

print(f"Sent {len(drink_refs)} orders to baristas")


2025-12-09 19:45:21,193	INFO logging.py:397 -- Registered dataset logger for dataset dataset_0_0
2025-12-09 19:45:21,201	INFO streaming_executor.py:682 -- [dataset]: A new progress UI is available. To enable, set `ray.data.DataContext.get_current().enable_rich_progress_bars = True` and `ray.data.DataContext.get_current().use_ray_tqdm = False`.


Processing orders...


Running 0: 0.00 row [00:00, ? row/s]2025-12-09 19:45:21,218	WARNING resource_manager.py:136 -- ⚠️  Ray's object store is configured to use only 15.5% of available memory (2.0GiB out of 12.9GiB total). For optimal Ray Data performance, we recommend setting the object store to at least 50% of available memory. You can do this by setting the 'object_store_memory' parameter when calling ray.init() or by setting the RAY_DEFAULT_OBJECT_STORE_MEMORY_PROPORTION environment variable.
2025-12-09 19:45:21,234	INFO streaming_executor.py:300 -- ✔️  Dataset dataset_0_0 execution finished in 0.00 seconds
✔️  Dataset dataset_0_0 execution finished in 0.00 seconds: : 6.00 row [00:00, 316 row/s]
2025-12-09 19:45:21,253	INFO util.py:257 -- Exiting prefetcher's background thread


Sent 6 orders to baristas


In [9]:
# Wait for all drinks to be ready (blocking)
completed_drinks = ray.get(drink_refs)

print("\nDrinks ready:")
for drink in completed_drinks:
    print(f"  {drink['drink']} - ${drink['price']:.2f} [{drink['status']}]")



Drinks ready:
  Cappuccino - $4.50 [ready]
  Latte - $5.00 [ready]
  Espresso - $3.00 [ready]
  Americano - $3.50 [ready]
  Mocha - $5.50 [ready]
  Flat White - $4.75 [ready]


In [10]:
# Record each sale in the cash register (actor)
print("\nRecording sales...")
sale_refs = []

for drink in completed_drinks:
    # Non-blocking: record the sale
    sale_ref = register.record_sale.remote(drink["price"])
    sale_refs.append(sale_ref)

# Wait for all sales to be recorded (blocking)
ray.get(sale_refs)
print("All sales recorded")



Recording sales...
All sales recorded


In [11]:
# Check today's totals
summary = ray.get(register.get_summary.remote())

print("\n" + "="*40)
print("DAILY SUMMARY")
print("="*40)
print(f"Drinks sold: {summary['drinks_sold']}")
print(f"Total revenue: ${summary['total_revenue']:.2f}")
print("="*40)



DAILY SUMMARY
Drinks sold: 6
Total revenue: $26.25


## What Just Happened

You built a distributed system without thinking about distributed systems. Ray handled the complexity:

- **Workers** (baristas) ran your tasks in parallel across available CPUs
- **Tasks** (`make_drink`) executed independently, scaling naturally with your worker pool
- **Actors** (`CashRegister`) maintained consistent state across concurrent operations
- **Ray Data** managed the input dataset, letting you iterate cleanly without manual partitioning
- **The cluster** (head node + workers) coordinated everything behind the scenes

The code looks almost sequential, but Ray parallelized aggressively wherever it could. Four baristas worked simultaneously. The cash register serialized updates to avoid conflicts. And you wrote maybe 50 lines of Python.

This pattern (stateless tasks for parallel work, stateful actors for shared data, datasets for structured input) scales from your laptop to hundreds of machines. The abstraction stays the same. Ray handles the distribution.


## Closing Time

When you're done, shut down the Ray cluster to free resources.


In [12]:
ray.shutdown()
print("Shop closed. See you tomorrow!")


Shop closed. See you tomorrow!


## Where to Go from Here

This coffee shop is tiny, but the pattern extends to real workloads:

**Bigger datasets:** Replace `rd.from_items()` with `rd.read_parquet()`, `rd.read_csv()`, or `rd.read_images()`. Ray Data can handle terabytes of data, streaming it through transformations without loading everything into memory.

**More complex tasks:** Your `@ray.remote` functions can do anything. Train models, process images, call APIs. If it's Python code that can run independently, it can be a Ray task.

**Distributed actors:** Actors can live on different machines, manage GPU resources, or coordinate complex workflows. You can even have multiple actors of the same class handling different shards of work.

**Multi-node clusters:** Instead of `ray.init(num_cpus=4)`, connect to a Ray cluster running across dozens of machines. Your code stays the same. Ray handles scheduling across nodes, moving data, and recovering from failures.

The coffee shop abstraction breaks down eventually, but the primitives don't. Tasks for stateless parallel work. Actors for stateful coordination. Datasets for distributed data. That's the core of Ray, and it's enough to build surprisingly complex systems.
